##### Copyright 2019 The TensorFlow Authors.



In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow ile egitimin dagitimi

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/tr/r1/tutorials/distribute/keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/tr/r1/tutorials/distribute/keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Note: Bu dökümanlar TensorFlow gönüllü kullanıcıları tarafından çevirilmiştir.
Topluluk tarafından sağlananan çeviriler gönüllülerin ellerinden geldiğince
güncellendiği için [Resmi İngilizce dökümanlar](https://www.tensorflow.org/?hl=en)
ile bire bir aynı olmasını garantileyemeyiz. Eğer bu tercümeleri iyileştirmek
için önerileriniz var ise lütfen [tensorflow/docs](https://github.com/tensorflow/docs)
havuzuna pull request gönderin. Gönüllü olarak çevirilere katkıda bulunmak için
[docs-tr@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-tr)
listesi ile iletişime geçebilirsiniz.

## Genel Bakis
`tf.distribute.Strategy` API'si TF programinizi birden cok bilgisayar/GPU'ya dagitmak icin bir taslak saglar. Buradaki amac kullanicilarin var olan modellerini ve kodlarini olabildigince ayni tutarak dagitim yapmalarini saglamaktir.

Bu rehber `tf.distribute.MirroredStrategy` taktigini kullanarak ayni makinedeki birden fazla GPU'da ayni "graph"i kullanarak eszamanli egitim yapmamizi saglar. Isin ozunde, bu taktik modelinizin degiskenlerini butun islemcilere kopyalar. Daha sonra, [all-reduce](http://mpitutorial.com/tutorials/mpi-reduce-and-allreduce/) yontemi ile butun islemcilerden gelen degisimleri birlestirir ve elde edilen bu degeri varolan butun kopyalara uygular.

`MirroredStrategy` taktigi TensorFlow'un icindeki dagitim taktiklerinden sadece biridir. Diger taktikler hakkinda bilgi edinmek icin bu rehbere goz atabilirsiniz: [dagitim taktikleri rehberi](../../guide/distribute_strategy.ipynb).


### Keras API
Bu ornekteki modeller ve egitim donguleri `tf.keras` API'si kullanilarak olusturulmustur. Eger, kendinize ozgu bir egitim dongusu olusturmak istiyorsaniz, [buraya](training_loops.ipynb) goz atiniz.

## Gereksinimleri indirelim

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
# TensorFlow kutuphanesini getirelim
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow.compat.v1 as tf

In [0]:
import tensorflow_datasets as tfds

import os

## Veri setini indirelim

MNIST veri setini [TensorFlow Datasets](https://www.tensorflow.org/datasets) kullanarak indirip yukleyelim.

`with_info` degiskenini `True` olarak belirtmek metadata'nin butun veri setini kapsadigini belirtir. Bu `metadata` `ds_info` degiskeninde saklanmaktadir. 
Bu metadata nesnesi egitim ve test orneklerinin sayisini da icinde barindirir.


In [0]:
datasets, ds_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = datasets['train'], datasets['test']

## Dagitim taktigimizi tanimlayalim


Bir `MirroredStrategy` nesnesi olusturalim. Bu sayede dagitimi kontrol edebilir ve modelimizi icinde olusturabilecegimiz bir ortam yonetmeni (`tf.distribute.MirroredStrategy.scope`) saglamis oluruz. 

In [0]:
strategy = tf.distribute.MirroredStrategy()

In [0]:
print ('Number of devices: {}'.format(strategy.num_replicas_in_sync))

## Input pipeline kuralim

Eger bir model birden fazla GPU ile egitiliyorsa, grup boyutleri ayni oranda arttirilmalidir ki islemci gucunu verimli sekilde kullanabilelim. Ayni zamanda ogrenme hizinin da GPU miktarina gore ayarlanmasi gerekir. 

In [0]:
# 'ds_info.splits.total_num_examples' metodunu kullanarak da orneklerin sayisini bulabilirsiniz.

num_train_examples = ds_info.splits['train'].num_examples
num_test_examples = ds_info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

Piksel degerleri, ki bunlar aslinda 0-255 arasindadir, [normallestirilerek 0 ile 1 arasinda bir degere indirgenmelidir](https://en.wikipedia.org/wiki/Feature_scaling). Bu ingirdenme olcegini bir fonksiyon ile tanimlayalim.

In [0]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

Simdi bu fonksiyonu egitim ve test verisine uygulayalim. Sonrasinda egitim verisini karistiralim ve [egitim icin gruplayalim](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch).


In [0]:
train_dataset = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

## Modeli olusturalim

Keras modelimizi `strategy.scope` ortaminda olusturalim ve toparlayalim.

In [0]:
with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(loss='sparse_categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

## Bildirim fonksiyonlarini tanimlayalim



Burada kullanilan bildirim fonksiyonlari sunlardir:
*   *Tensorboard*: Bu geri arama Tensorboard'lar icin kayit tutar. Bu kayitlar ile bir grafik olusturabiliriz.
*   *Model Checkpoint*: Bu geri arama her devir sonunda modeli kaydeder.
*   *Learning Rate Scheduler*: Bu geri aramayi kullanarak, ogrenme hizinin her devir ya da gruptan sonra degismesini programlayabilirsiniz.

Daha aciklayici olmasi icin, 'ogrenme hizini' bir geri arama ile kitapciga yazdirabilirsiniz.

In [0]:
# Kontrol noktalarini kaydetmek icin bir kontrol noktasi dosyasi olusturalim.

checkpoint_dir = './training_checkpoints'
# Kontrol noktasi dosyalarinin ismi
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [0]:
# Azalan ogrenme hizi icin bir fonksiyon
# Ihtiyaciniz olan butun azalma fonksiyonlarini kullanabilirsiniz.
def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

In [0]:
# Her devrin sonunda LR'i (ogrenme hizini) yazdirmak icin bir geri arama fonksiyonu.
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print ('\nLearning rate for epoch {} is {}'.format(
        epoch + 1, tf.keras.backend.get_value(model.optimizer.lr)))

In [0]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

## Egitim ve yorumlama

Simdi, modelimizi her zamanki gibi egitelim: `fit` yontemini model uzerinde cagirip bu rehberin basinda olusturdugumuz veri setini modele verelim. Bu asama dagitim yapsaniz da yapmasaniz da hep ayni kalacaktir.


In [0]:
model.fit(train_dataset, epochs=10, callbacks=callbacks)

Asagida gordugunuz gibi, kontrol noktalari hafizaya kaydedilmektedir.

In [0]:
# Kontrol noktalari dosyasina bakalim.
!ls {checkpoint_dir}

Modelin nasil calistigini gormek icin, en son kaydedilen kontrol noktasini yukleyip 'evaluate' yontemini test verisinde cagirabilirsiniz.

'evaluate' yontemini daha once yaptiginiz gibi uygun veri setlerinde kullanmalisiniz.

In [0]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

eval_loss, eval_acc = model.evaluate(eval_dataset)
print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

Programin sonuclarini, TensorBoard kayitlarini indirerek terminalde gorebilirsiniz. 

```
$ tensorboard --logdir=path/to/log-directory
```

In [0]:
!ls -sh ./logs

## Kaydedilen modelin _'SavedModel'_ cikartilmasi

Eger "graph"larin ve degiskenlerin program disinda kullanilmasini istiyorsaniz, `SavedModel` sizin icin ideal yontem. Bu yontem herhangi bir kapsama bagli olmadan yuklenebir ve herhangi bir platforma da bagli degildir.

In [0]:
path = 'saved_model/'

In [0]:
tf.keras.experimental.export_saved_model(model, path)

Modeli 'strategy.scope' olmadan yukleyelim.

In [0]:
unreplicated_model = tf.keras.experimental.load_from_saved_model(path)

unreplicated_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'])

eval_loss, eval_acc = unreplicated_model.evaluate(eval_dataset)
print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

## Sirada ne var?

Dagitim taktikleri rehberini [distribution strategy guide](../../guide/distribute_strategy_tf1.ipynb) okuyunuz.

Note: `tf.distribute.Strategy` surekli gelistirilmektedir ve yakin zamanda yeni ornekler ve rehberler buraya eklenecektir. Lutfen bu yontemleri deneyiniz. Sizden gelen yorumlara her zaman acigiz. Bu yorumlari buraya yazabilirsiniz [issues on GitHub](https://github.com/tensorflow/tensorflow/issues/new).